<a href="https://colab.research.google.com/github/bhonsleaditya1/Lending-Club-PySpark/blob/master/Lending_Club_XGBoost_1vR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install and Import Libraries

In [ ]:
!pip install -q bayesian-optimization
!pip install -q scikit-optimize

In [ ]:
import os,gc,re,matplotlib,joblib,ast
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection  import StratifiedKFold,train_test_split,cross_validate
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler,LabelEncoder,LabelBinarizer
from sklearn.metrics import roc_auc_score,auc,roc_curve,accuracy_score,classification_report,confusion_matrix,f1_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import xgboost as xgb
from scipy.stats import zscore
from sklearn import model_selection 
from bayes_opt import BayesianOptimization
from sklearn.cluster import DBSCAN
from sklearn.multiclass import OneVsRestClassifier

#Import Data

In [ ]:
pdf = pd.read_csv('/content/drive/My Drive/Lending-Club/loanFinal.csv', header=0, escapechar='\\')
dropcol = pd.read_csv('/content/drive/My Drive/Lending-Club/FinalDrop.csv',header=None)[0].to_list()
dbindex = pd.read_csv('/content/drive/My Drive/Lending-Club/DBRemove.csv')
pdf = pdf[pdf.amnt_left_per<=100]
pdf = pdf.drop(dropcol,axis=1)
pdf = pdf.drop(dbindex['index'].to_list())
pdf = pdf.reset_index().drop('index',axis=1)
pdf.count()
pdf['year'] = pd.DatetimeIndex(pdf['issue_d']).year
reg = pdf.select_dtypes(['float64']).columns
clas = pdf.select_dtypes(['O']).columns

#Pre-Process Data

##OneHotEncoding

In [ ]:
#OneHotEncoding 1 vs R 
pdf.loc[pdf['target']==2,'target']=0
for i in clas:
  if pdf[i].nunique()<25:
    l = pd.get_dummies(pdf[i], prefix=i)
    for j in l.columns:
      pdf[j] = l[j]
for i in clas:
  pdf = pdf.drop(i,axis=1)
pdf = pdf.rename(columns={'emp_length_< 1 year':'emp_length_less 1 year'})
timetest = pdf[pdf.year==2016]
train = pdf[pdf.year!=2016]
timetest = timetest.drop(['year'],axis=1)
train = train.drop(['year'],axis=1)
y_timetest=pd.DataFrame()
y_t = pd.DataFrame()
y_timetest['target'] = timetest['target']
y_t['target'] = train['target']
timetest = timetest.drop('target',axis=1)
train = train.drop('target',axis=1)
X_train, X_test, y_train, y_test = train_test_split(train, y_t, random_state=1301, test_size=0.3)
scale = StandardScaler()
scale.fit(X_train)
X_train = scale.transform(X_train)
X_test = scale.transform(X_test)
timetest = scale.transform(timetest)

#From this Cell
1.   1 is Class 1
2.   0 is Rest



#Training Base XGBoost Model 

In [ ]:
def gini_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc-1
    err = gini
    return 'gini_err', err
clf = xgb.XGBClassifier(tree_method='gpu_hist',objective='binary:logistic',num_boost_round=70,verbose_eval=False,n_jobs=-1)
clf.fit(X_train,y_train['target'],eval_metric=gini_eval,eval_set=[(X_train,y_train['target']),(timetest,y_timetest['target'])],early_stopping_rounds=50)

#Recursive Feature Selection with CV 

In [ ]:
classifier = xgb.XGBClassifier(objective= 'binary:logistic',tree_method='gpu_hist')

selector = RFECV(estimator=classifier,cv=StratifiedKFold(n_splits=5,random_state=1301, shuffle=True), scoring='f1_weighted')
selector.fit(X_train.fillna(-999999), y_train)

print('The optimal number of features is {}'.format(selector.n_features_))
features = [f for f,s in zip(X_train.columns, selector.support_) if s]
print('The selected features are:')
print ('{}'.format(features))

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.savefig('feature_auc_nselected.png', bbox_inches='tight', pad_inches=1)

##!!!Uncomment **ONLY** when you want to use RFECV features!!!

In [ ]:
#X_train = X_train[features]
#X_test = X_test[features]
#timetest = timetest[features]

#Hyperparameter Tuning

##Bayesian Optimization

In [ ]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(timetest,label=y_timetest)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
cvvalues = []
evalsdict={}
ginivalues=[]
#Bayesian Optimization function for xgboost
def gini_eval(y_pred, dtrain):
    err=0
    y_true = dtrain.get_label()
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc -1
    err=1-gini
    gc.collect()
    return 'gini_err', err
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth,gamma,n_estimators,learning_rate,reg_alpha,scale_pos_weight,colsamplebytree,max_delta_step):
  global cvvalues,ginivalues,X_train,y_train,timetest,y_timetest
  dicct={}
  params = {'max_depth': int(max_depth),
          'gamma': gamma,
          'n_estimators': int(n_estimators),
          'learning_rate':learning_rate,
          'subsample': 0.8,
          'eta': 0.1,
          'reg_alpha':reg_alpha,
          'scale_pos_weight':scale_pos_weight,
          'colsamplebytree':colsamplebytree,
          'eval_metric': 'error',
          'objective': 'binary:logistic',
          'max_delta_step':max_delta_step,
          'tree_method':'gpu_hist'
          }
  #Cross validating with the specified parameters in 5 folds and 70 iterations
  #cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5,
                     #feval=gini_eval
                     #metrics='logloss')
  clf = xgb.XGBClassifier(**params,n_jobs=-1)
  clf.fit(X_train,y_train['target'],eval_metric=gini_eval,eval_set=[(X_train,y_train['target']),(X_test,y_test['target'])],early_stopping_rounds=50,verbose=False)
  #Return the gini
  cvvalues.append(clf.best_score)
  return 1-clf.best_score

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                            'gamma': (0, 1),
                                            'scale_pos_weight': (7,100),                                          
                                            'n_estimators':(100,1000),
                                            'reg_alpha': (0,1),
                                            'learning_rate': (0.01,0.5),
                                            'colsamplebytree':(0,1),
                                            'max_delta_step':(0,1)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=5, init_points=5, acq='ei')

### Saving all Parameters



In [ ]:
pd.DataFrame.from_dict(xgb_bo.res).to_excel('/content/drive/My Drive/Lending-Club/1vR_Bayesian_XGB_params.xlsx',index=False)

###Gini and f1 for above generated model parameters

In [ ]:
gini={}
gini_train={}
gini_time={}
f1={}
f1_train={}
f1_time={}
p = pd.DataFrame.from_dict(xgb_bo.res)
for i in range(10):
  para=p['params'][i]
  para['n_estimators']=int(para['n_estimators'])
  para['max_depth']=int(para['max_depth'])
  para['scale_pos_weight']=int(para['scale_pos_weight'])
  para['objective'] = 'binary:logistic'
  para['tree_method'] = 'gpu_hist'
  #dtrain = xgb.DMatrix(X_train,label=y_train)
  clf = xgb.XGBClassifier(**para).fit(X_train,y_train)
  res = clf.predict_proba(X_test)
  gini[i]=calc_gini(y_test,res,2)
  res = clf.predict(X_test)
  f1[i]=f1_score(y_test,res,average=None)
  res = clf.predict_proba(X_train)
  gini_train[i]=calc_gini(y_train,res,2)
  res = clf.predict(X_train)
  f1_train[i]=f1_score(y_train,res,average=None)
  res = clf.predict_proba(timetest)
  gini_time[i]=calc_gini(y_timetest,res,2)
  res = clf.predict(timetest)
  f1_time[i]=f1_score(y_timetest,res,average=None)
  gc.collect()
  #fscores = pd.DataFrame({'X': list(clf.get_fscore().keys()), 'Y': list(clf.get_fscore().values())})
  #plt.figure(i)
  #fscores.sort_values(by='Y')[:20].plot.bar(x='X')
#result = clf.predict_proba(X_test)

###Compile Results

In [ ]:
#Training data
gini_train

In [ ]:
#Test data
gini

In [ ]:
#Time data
gini_time

In [ ]:
#Training data
f1_train

In [ ]:
#Test data
f1

In [ ]:
#Time data
f1_time

###Training and Saving Best Model

In [ ]:
para= xgb_bo.max['params']
para['n_estimators']=int(para['n_estimators'])
para['max_depth']=int(para['max_depth'])
para['scale_pos_weight']=int(para['scale_pos_weight'])
para['objective'] = 'binary:logistic'
para['tree_method'] ='gpu_hist'
clf = xgb.XGBClassifier(**para,n_jobs=-1)
clf.fit(X_train,y_train['target'],eval_metric=gini_eval,eval_set=[(X_train,y_train['target']),(X_test,y_test['target'])],early_stopping_rounds=50,verbose=False)
joblib.dump(clf, '/content/drive/My Drive/Lending-Club/Models/XGB_1_Bayesian_best.pkl')

##RandomSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
bayes_cv_tuner = RandomizedSearchCV(estimator = xgb.XGBClassifier(
                                n_jobs = -1,
                                objective = 'binary:logistic',
                                learning_rate = 0.08,
                                silent=1,
                                n_estimators = 1000,
                                tree_method='gpu_hist',                                
                                early_stopping_rounds=50
                                ),
    param_distributions = {
        'max_depth': (6, 8),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-2, 1000, 'log-uniform'),
        'reg_alpha': (1e-2, 1.0, 'log-uniform'),
        'gamma': (1e-2, 0.5, 'log-uniform'),
        'min_child_weight': (0, 20),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42),
    n_jobs = 8,
    n_iter = 300,   
    verbose = 500,
    refit = True,
    random_state = 12345)

#Saving all results
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    )) 
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name + "1vR_Random_cv_results.csv")

#Best model
result = bayes_cv_tuner.fit(X_train, y_train)

#Analysing Model

##Input Model

In [ ]:
model = #input the model you want to analyse

##Percentile f1 score

In [ ]:
from sklearn.metrics import f1_score
res = model.predict_proba(X_train)
th1=1-(y_train.target==0).sum()/y_train.target.count()*1
th2=1-(y_train.target==1).sum()/y_train.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_train==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_train==1,1,0)))
res = model.predict_proba(X_test)
th1=1-(y_test.target==0).sum()/y_test.target.count()*1
th2=1-(y_test.target==1).sum()/y_test.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_test==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_test==1,1,0)))
res = model.predict_proba(timetest)
th1=1-(y_timetest.target==0).sum()/y_timetest.target.count()*1
th2=1-(y_timetest.target==1).sum()/y_timetest.target.count()*1
c1=pd.DataFrame(res[:,0])
c2=pd.DataFrame(res[:,1])
print(f1_score(np.where(c1>c1.quantile(th1),1,0),np.where(y_timetest==0,1,0)))
print(f1_score(np.where(c2>c2.quantile(th2),1,0),np.where(y_timetest==1,1,0)))

##Gini per Class

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
gini =0
#No of classes is j
j=2
y_predtrain=model.predict_proba(X_train)
y_pred=model.predict_proba(X_test)
y_predtime=model.predict_proba(timetest)
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_test))
y_gini = drop_enc.transform(pd.DataFrame(y_test)).toarray()
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_timetest))
y_ginitime = drop_enc.transform(pd.DataFrame(y_timetest)).toarray()
drop_enc = OneHotEncoder().fit(pd.DataFrame(y_train))
y_ginitrain = drop_enc.transform(pd.DataFrame(y_train)).toarray()
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_ginitrain[:, i], y_predtrain[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_gini[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)
for i in range(j):
    fpr[i], tpr[i], _ = roc_curve(y_ginitime[:, i], y_predtime[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(2*roc_auc[i]-1)

##Accuracy

In [ ]:
print(accuracy_score(y_train,model.predict(X_train)))
print(accuracy_score(y_test,model.predict(X_test)))
print(accuracy_score(y_timetest,model.predict(timetest)))

##Classification Report

In [ ]:
y_predtrain = model.predict(X_train)
y_pred = model.predict(X_test)
y_predtime = model.predict(timetest)
print(classification_report(y_train, y_predtrain))
print(classification_report(y_test, y_pred))  
print(classification_report(y_timetest, y_predtime))

#Feature Importance

##Input Model

In [ ]:
model_f = #input the model you want to find feature Importance
model_name = #input model name

##Excel of Feature Importance

In [ ]:
pd.DataFrame(dict(zip(X_train,model_f.feature_importances_)).items()).to_excel(str(model_name)+'_1vR_feature_imp.xlsx')